In [ ]:
"""
Concrete Compressive Strength Prediction
Complete Training Pipeline Notebook

This notebook demonstrates the entire workflow:
1. Data Loading & Exploration
2. Data Cleaning & Preprocessing
3. Feature Engineering
4. Model Training & Evaluation
5. Model Deployment Preparation
"""

# %% [markdown]
# # Concrete Quality Prediction - Complete Training Pipeline

# %% [markdown]
# ## 1. Setup and Imports

# %%
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
import xgboost as xgb
import joblib
import warnings

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ All libraries imported successfully")

# %% [markdown]
# ## 2. Data Loading

# %%
# Load the UCI Concrete dataset
# Download from: https://archive.ics.uci.edu/ml/datasets/Concrete+Compressive+Strength

df = pd.read_excel('data/raw/Concrete_Data.xls')

# Rename columns for clarity
df.columns = [
    'cement', 'blast_furnace_slag', 'fly_ash', 'water',
    'superplasticizer', 'coarse_aggregate', 'fine_aggregate',
    'age', 'compressive_strength'
]

print(f"Dataset shape: {df.shape}")
print(f"\nFirst few rows:")
display(df.describe())

# Check for missing values
print("\nMissing values:")
print(df.isnull().sum())

# Check for duplicates
print(f"\nDuplicate rows: {df.duplicated().sum()}")

# %%
# Distribution of target variable
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram
axes[0].hist(df['compressive_strength'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Compressive Strength (MPa)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Compressive Strength')
axes[0].axvline(df['compressive_strength'].mean(), color='red', linestyle='--', label='Mean')
axes[0].legend()

# Box plot
axes[1].boxplot(df['compressive_strength'])
axes[1].set_ylabel('Compressive Strength (MPa)')
axes[1].set_title('Box Plot of Compressive Strength')

plt.tight_layout()
plt.show()

print(f"Mean strength: {df['compressive_strength'].mean():.2f} MPa")
print(f"Median strength: {df['compressive_strength'].median():.2f} MPa")
print(f"Std deviation: {df['compressive_strength'].std():.2f} MPa")

# %%
# Correlation matrix
plt.figure(figsize=(12, 10))
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1)
plt.title('Correlation Matrix - Concrete Components vs Strength')
plt.tight_layout()
plt.show()

# Most correlated features with strength
strength_corr = correlation_matrix['compressive_strength'].sort_values(ascending=False)
print("\nCorrelation with Compressive Strength:")
print(strength_corr)

# %%
# Feature distributions
features = ['cement', 'water', 'age', 'superplasticizer']
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.ravel()

for idx, feature in enumerate(features):
    axes[idx].scatter(df[feature], df['compressive_strength'], alpha=0.5)
    axes[idx].set_xlabel(feature.replace('_', ' ').title())
    axes[idx].set_ylabel('Compressive Strength (MPa)')
    axes[idx].set_title(f'{feature.replace("_", " ").title()} vs Strength')
    
    # Add trend line
    z = np.polyfit(df[feature], df['compressive_strength'], 1)
    p = np.poly1d(z)
    axes[idx].plot(df[feature], p(df[feature]), "r--", alpha=0.8, linewidth=2)

plt.tight_layout()
plt.show()

# %% [markdown]
# ## 4. Feature Engineering

# %%
def engineer_features(df):
    """Create derived features"""
    df_feat = df.copy()
    
    # Water-cement ratio (critical for strength)
    df_feat['water_cement_ratio'] = df_feat['water'] / (df_feat['cement'] + 1e-6)
    
    # Total cementitious materials
    df_feat['total_cementitious'] = (df_feat['cement'] + 
                                      df_feat['blast_furnace_slag'] + 
                                      df_feat['fly_ash'])
    
    # Total aggregates
    df_feat['total_aggregate'] = (df_feat['coarse_aggregate'] + 
                                   df_feat['fine_aggregate'])
    
    # Fine to coarse aggregate ratio
    df_feat['fine_coarse_ratio'] = (df_feat['fine_aggregate'] / 
                                     (df_feat['coarse_aggregate'] + 1e-6))
    
    # Superplasticizer per unit cement
    df_feat['sp_cement_ratio'] = (df_feat['superplasticizer'] / 
                                   (df_feat['cement'] + 1e-6))
    
    # Age transformations
    df_feat['age_log'] = np.log1p(df_feat['age'])
    df_feat['age_sqrt'] = np.sqrt(df_feat['age'])
    df_feat['is_early_age'] = (df_feat['age'] <= 7).astype(int)
    df_feat['is_mature'] = (df_feat['age'] >= 28).astype(int)
    
    # SCM replacement percentage
    df_feat['replacement_pct'] = ((df_feat['blast_furnace_slag'] + df_feat['fly_ash']) / 
                                   (df_feat['total_cementitious'] + 1e-6))
    
    # Interaction features
    df_feat['cement_age'] = df_feat['cement'] * df_feat['age_log']
    df_feat['water_age'] = df_feat['water'] * df_feat['age_log']
    
    return df_feat

# Apply feature engineering
df_engineered = engineer_features(df)
print(f"Original features: {df.shape[1]}")
print(f"After engineering: {df_engineered.shape[1]}")
print(f"\nNew features created: {df_engineered.shape[1] - df.shape[1]}")

# Display new features
new_features = [col for col in df_engineered.columns if col not in df.columns]
print("\nNew features:")
for feat in new_features:
    print(f"  - {feat}")

# %% [markdown]
# ## 5. Data Preprocessing

# %%
# Separate features and target
X = df_engineered.drop('compressive_strength', axis=1)
y = df_engineered['compressive_strength']

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"\nTraining set: {X_train.shape}")
print(f"Test set: {X_test.shape}")

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrame for clarity
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

print("\n✓ Data preprocessing completed")

# %% [markdown]
# ## 6. Model Training

# %%
# Initialize models
models = {
    'Linear Regression': Ridge(alpha=1.0),
    'Random Forest': RandomForestRegressor(
        n_estimators=200,
        max_depth=20,
        min_samples_split=5,
        random_state=42,
        n_jobs=-1
    ),
    'XGBoost': xgb.XGBRegressor(
        n_estimators=300,
        max_depth=7,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1
    ),
    'Gradient Boosting': GradientBoostingRegressor(
        n_estimators=200,
        max_depth=5,
        learning_rate=0.05,
        random_state=42
    ),
    'Neural Network': MLPRegressor(
        hidden_layer_sizes=(128, 64, 32),
        activation='relu',
        solver='adam',
        max_iter=500,
        random_state=42
    )
}

# Train and evaluate models
results = {}

print("=" * 60)
print("TRAINING MODELS")
print("=" * 60)

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Train
    model.fit(X_train_scaled, y_train)
    
    # Predict
    y_pred_train = model.predict(X_train_scaled)
    y_pred_test = model.predict(X_test_scaled)
    
    # Evaluate
    train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    test_mae = mean_absolute_error(y_test, y_pred_test)
    
    # Cross-validation
    cv_scores = cross_val_score(model, X_train_scaled, y_train, 
                                 cv=5, scoring='r2')
    
    results[name] = {
        'model': model,
        'train_rmse': train_rmse,
        'test_rmse': test_rmse,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'test_mae': test_mae,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'predictions': y_pred_test
    }
    
    print(f"  Train RMSE: {train_rmse:.3f} MPa")
    print(f"  Test RMSE: {test_rmse:.3f} MPa")
    print(f"  Test R²: {test_r2:.4f}")
    print(f"  Test MAE: {test_mae:.3f} MPa")
    print(f"  CV R² (mean ± std): {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

# %% [markdown]
# ## 7. Model Comparison

# %%
# Create comparison DataFrame
comparison_df = pd.DataFrame({
    'Model': list(results.keys()),
    'Test RMSE': [r['test_rmse'] for r in results.values()],
    'Test R²': [r['test_r2'] for r in results.values()],
    'Test MAE': [r['test_mae'] for r in results.values()],
    'CV R² Mean': [r['cv_mean'] for r in results.values()]
}).sort_values('Test R²', ascending=False)

print("\n" + "=" * 60)
print("MODEL COMPARISON")
print("=" * 60)
display(comparison_df)

# Visualize comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# RMSE comparison
axes[0].barh(comparison_df['Model'], comparison_df['Test RMSE'], color='skyblue')
axes[0].set_xlabel('RMSE (MPa)')
axes[0].set_title('Model Comparison - RMSE (lower is better)')
axes[0].invert_yaxis()

# R² comparison
axes[1].barh(comparison_df['Model'], comparison_df['Test R²'], color='lightgreen')
axes[1].set_xlabel('R² Score')
axes[1].set_title('Model Comparison - R² (higher is better)')
axes[1].invert_yaxis()

# MAE comparison
axes[2].barh(comparison_df['Model'], comparison_df['Test MAE'], color='salmon')
axes[2].set_xlabel('MAE (MPa)')
axes[2].set_title('Model Comparison - MAE (lower is better)')
axes[2].invert_yaxis()

plt.tight_layout()
plt.show()

# %%
# Best model
best_model_name = comparison_df.iloc[0]['Model']
best_model = results[best_model_name]['model']
print(f"\n🏆 Best Model: {best_model_name}")
print(f"   Test R²: {results[best_model_name]['test_r2']:.4f}")
print(f"   Test RMSE: {results[best_model_name]['test_rmse']:.3f} MPa")

# %% [markdown]
# ## 8. Prediction Analysis

# %%
# Actual vs Predicted plot for best model
y_pred_best = results[best_model_name]['predictions']

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Scatter plot
axes[0].scatter(y_test, y_pred_best, alpha=0.6, edgecolors='k')
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
             'r--', lw=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual Strength (MPa)')
axes[0].set_ylabel('Predicted Strength (MPa)')
axes[0].set_title(f'{best_model_name}: Actual vs Predicted')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Residual plot
residuals = y_test - y_pred_best
axes[1].scatter(y_pred_best, residuals, alpha=0.6, edgecolors='k')
axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('Predicted Strength (MPa)')
axes[1].set_ylabel('Residuals (MPa)')
axes[1].set_title(f'{best_model_name}: Residual Plot')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Error distribution
plt.figure(figsize=(10, 5))
plt.hist(residuals, bins=50, edgecolor='black', alpha=0.7)
plt.axvline(x=0, color='r', linestyle='--', linewidth=2, label='Zero Error')
plt.xlabel('Prediction Error (MPa)')
plt.ylabel('Frequency')
plt.title(f'{best_model_name}: Distribution of Prediction Errors')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Mean Absolute Error: {np.abs(residuals).mean():.3f} MPa")
print(f"Error Std Dev: {residuals.std():.3f} MPa")

# %% [markdown]
# ## 9. Feature Importance Analysis

# %%
# Feature importance for tree-based models
if best_model_name in ['Random Forest', 'XGBoost', 'Gradient Boosting']:
    importances = best_model.feature_importances_
    feature_importance_df = pd.DataFrame({
        'Feature': X_train.columns,
        'Importance': importances
    }).sort_values('Importance', ascending=False)
    
    # Plot top 15 features
    plt.figure(figsize=(12, 8))
    top_features = feature_importance_df.head(15)
    plt.barh(top_features['Feature'], top_features['Importance'], color='steelblue')
    plt.xlabel('Importance Score')
    plt.title(f'{best_model_name}: Top 15 Feature Importances')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    print("\nTop 10 Most Important Features:")
    display(feature_importance_df.head(10))

# %% [markdown]
# ## 10. Hyperparameter Tuning (XGBoost Example)

# %%
print("Performing hyperparameter tuning for XGBoost...")
print("This may take several minutes...\n")

# Define parameter grid
param_grid = {
    'n_estimators': [200, 300, 400],
    'max_depth': [5, 7, 9],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.7, 0.8, 0.9]
}

# Grid search
xgb_model = xgb.XGBRegressor(random_state=42, n_jobs=-1)
grid_search = GridSearchCV(
    xgb_model, 
    param_grid, 
    cv=5, 
    scoring='r2',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train_scaled, y_train)

print("\n" + "=" * 60)
print("HYPERPARAMETER TUNING RESULTS")
print("=" * 60)
print(f"\nBest parameters: {grid_search.best_params_}")
print(f"Best CV R²: {grid_search.best_score_:.4f}")

# Evaluate tuned model
tuned_model = grid_search.best_estimator_
y_pred_tuned = tuned_model.predict(X_test_scaled)
tuned_r2 = r2_score(y_test, y_pred_tuned)
tuned_rmse = np.sqrt(mean_squared_error(y_test, y_pred_tuned))

print(f"\nTuned Model Performance:")
print(f"  Test R²: {tuned_r2:.4f}")
print(f"  Test RMSE: {tuned_rmse:.3f} MPa")

# %% [markdown]
# ## 11. Save Models

# %%
import os

# Create models directory
os.makedirs('models', exist_ok=True)

# Save best model
joblib.dump(best_model, f'models/{best_model_name.lower().replace(" ", "_")}.pkl')
print(f"✓ Saved {best_model_name}")

# Save tuned XGBoost
joblib.dump(tuned_model, 'models/xgboost_tuned.pkl')
print("✓ Saved Tuned XGBoost")

# Save scaler
joblib.dump(scaler, 'models/scaler.pkl')
print("✓ Saved Scaler")

# Save feature names
feature_names = X_train.columns.tolist()
joblib.dump(feature_names, 'models/feature_names.pkl')
print("✓ Saved Feature Names")

# Save model metadata
metadata = {
    'best_model': best_model_name,
    'test_r2': results[best_model_name]['test_r2'],
    'test_rmse': results[best_model_name]['test_rmse'],
    'test_mae': results[best_model_name]['test_mae'],
    'feature_count': len(feature_names),
    'training_samples': len(X_train)
}
joblib.dump(metadata, 'models/metadata.pkl')
print("✓ Saved Metadata")

print("\n✅ All models and artifacts saved successfully!")

# %% [markdown]
# ## 12. Production Readiness Check

# %%
print("=" * 60)
print("PRODUCTION READINESS CHECKLIST")
print("=" * 60)

checklist = {
    "Model trained": True,
    "Model evaluated": True,
    "Feature engineering documented": True,
    "Models saved": True,
    "Scaler saved": True,
    "Feature names saved": True,
    "Test R² > 0.85": results[best_model_name]['test_r2'] > 0.85,
    "Test RMSE < 6 MPa": results[best_model_name]['test_rmse'] < 6,
    "No data leakage": True,
    "Cross-validation performed": True
}

for item, status in checklist.items():
    icon = "✅" if status else "❌"
    print(f"{icon} {item}")

if all(checklist.values()):
    print("\n🎉 Model is READY for production deployment!")
else:
    print("\n⚠️  Some items need attention before deployment")

# %% [markdown]
# ## 13. Example Predictions

# %%
print("\n" + "=" * 60)
print("EXAMPLE PREDICTIONS")
print("=" * 60)

# Example concrete mixes
examples = [
    {
        'name': 'Low Strength Mix',
        'cement': 200, 'blast_furnace_slag': 30, 'fly_ash': 0,
        'water': 190, 'superplasticizer': 2,
        'coarse_aggregate': 1050, 'fine_aggregate': 800, 'age': 28
    },
    {
        'name': 'Standard Mix',
        'cement': 350, 'blast_furnace_slag': 70, 'fly_ash': 0,
        'water': 175, 'superplasticizer': 6,
        'coarse_aggregate': 1000, 'fine_aggregate': 750, 'age': 28
    },
    {
        'name': 'High Strength Mix',
        'cement': 450, 'blast_furnace_slag': 100, 'fly_ash': 50,
        'water': 150, 'superplasticizer': 12,
        'coarse_aggregate': 950, 'fine_aggregate': 700, 'age': 28
    }
]

for example in examples:
    name = example.pop('name')
    
    # Create DataFrame
    ex_df = pd.DataFrame([example])
    
    # Engineer features
    ex_feat = engineer_features(ex_df)
    
    # Ensure all features present
    for col in X_train.columns:
        if col not in ex_feat.columns:
            ex_feat[col] = 0
    
    ex_feat = ex_feat[X_train.columns]
    
    # Scale
    ex_scaled = scaler.transform(ex_feat)
    
    # Predict
    prediction = best_model.predict(ex_scaled)[0]
    
    print(f"\n{name}:")
    print(f"  Cement: {example['cement']} kg/m³")
    print(f"  Water: {example['water']} kg/m³")
    print(f"  Age: {example['age']} days")
    print(f"  → Predicted Strength: {prediction:.2f} MPa")

print("\n" + "=" * 60)
print("✅ Training pipeline completed successfully!")
print("=" * 60)head())

# %% [markdown]
# ## 3. Exploratory Data Analysis

# %%
# Basic statistics
print("=" * 60)
print("DATASET STATISTICS")
print("=" * 60)
display(df.